# 6.1卷积神经网络的原则
1.平移不变性(translation invariance)
    表示不管检测对象出现在图像中的哪个位置，神经网络的前面几层应该对相同的图像区域具有相似的翻译，即“平移不变性”
2.局部性(locality)
    神经网络的前面几层应该只探索输入图像中的局部区域，而不过多在意图像中相隔较远区域的关系，即“局部性”原则

# 6.2图像卷积
## 6.2.1互相关运算
实际上，卷积层进行的是互相关运算，而不是卷积运算

对互相关运算简化处理，忽略通道（第三维）的情况，我们关注输入的图像大小为n_h和n_w，卷积核的大小k_h和k_w，则输出大小为:

(n_h-k_h+1)*(n_w-k_w+1)

注意到，输出的维度会比输入的维度小，那么如果叠加多个像上面的卷积层，输出的维度会逐渐缩小至消失，因此会在图像边缘加入“填充”(padding)来保持输出大小不变。

 为代码实现（主要用到corr2d函数）

In [1]:
import torch
from torch import nn
from d2l import torch as d2l
def corr2d(X,K): #@save
    '''二维互相关运算'''
    h,w = K.shape      #卷积核的高和宽
    Y = torch.zeros((X.shape[0]-h+1,X.shape[1]-w+1))  #初始化输出的张量
    for i in range(Y.shape[0]):    # 双循环计算输出的张量中每个元素
        for j in range(Y.shape[1]):
            Y[i,j] = (X[i:i+h,j:j+w]*K).sum()
    return Y

X = torch.tensor([[0.0,1.0,2.0],[3.0,4.0,5.0],[6.0,7.0,8.0]])
K = torch.tensor([[0.1,1.0],[2.0,3.0]])
corr = corr2d(X,K)
print(corr)

tensor([[19.0000, 25.1000],
        [37.3000, 43.4000]])


注意到上述代码实际运行时间约13秒，而输入与核的大小均很小，因此双循环的方式实现corr2d时间复杂度高，向量化计算会更快


In [ ]:
class Conv2D(nn.Module):
    def __init__(self,kernel_size):
        super().__init()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bais = nn.Parameter(torch.zeros(1))

    def forward(self,x):
        return corr2d(x,self.weight) + self.bais

## 6.2.3图像中目标的边缘检测
卷积层的简单应用：通过找到像素变化的位置来检测图像中不同颜色的边缘

1.构造一个6像素x8像素的黑白图像，中间4列为黑色，其余像素为白色

In [3]:
X = torch.ones((6,8))
X[:,2:6] = 0
print(X)

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])


2.构造高度为1，宽度为2的卷积核K。互相关运算时，如果水平相邻的两元素相同，则输出为0，否则输出为非0

In [5]:
K = torch.tensor([[1.0,-1.0]])

3.执行互相关运算

In [6]:
Y = corr2d(X,K)
print(Y)

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])


将二维图像转置，进行互相关运算

In [7]:
corr2d(X.t(),K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

这个卷积核K只能检测垂直边缘，无法检测水平边缘

## 6.2.4卷积核
学习由X生成Y的卷积核，暂且忽略偏置项

In [9]:
# 构造一个二维卷积层，它具有1个输出通道和形状为(1,2)的卷积核
conv2d = nn.Conv2d(1,1,kernel_size=(1,2),bias=False)

# 这个二维卷积层使用四维输入和输出格式(批量大小、通道、高度、宽度)
X=X.reshape((1,1,6,8))
Y=Y.reshape((1,1,6,7))
lr = 3e-2 #学习率

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y)**2
    conv2d.zero_grad()
    l.sum().backward()
    # 迭代卷积核
    conv2d.weight.data[:] -= lr*conv2d.weight.grad
    if (i+1) % 2 ==0:
        print(f'epoch {i+1},loss {l.sum():.3f}')

epoch 2,loss 7.283
epoch 4,loss 2.464
epoch 6,loss 0.922
epoch 8,loss 0.363
epoch 10,loss 0.146


解释一下这里的conv2d.zero_grad()的作用是将卷积层conv2d的所有参数的梯度设置为0，原因是当调用backward()方法进行反向传播计算的时候，梯度值会累加到参数的.grad属性中，会与之前得到的梯度相加，所以需要置0

In [12]:
conv2d.weight.data.reshape((1,2))

tensor([[ 0.9531, -1.0316]])

## 6.2.5互相关和卷积
由于卷积核是从数据中学习的，因此无论执行严格的卷积运算还是互相关运算，卷积层的输出均不受影响。

为了与文献标准术语保持一致，我们将“互相关运算”称为卷积运算；对于卷积核张量上的权重，我们称其为元素

## 6.2.6特征映射和感受野
卷积层有时成为特征映射(feature map)，可以被视为一个输入映射到下一层的空间维度的转换器

在CNN中，对于某一层的任意元素x，其感受野(receptive field)指在向前传播期间可能影响到x计算的所有元素（来自所有之前层）

感受野的图解在Ob/深度学习/L7/P46

# 6.3填充和步幅
## 6.3.1填充
在应用多层卷积时，我们尝尝丢失边缘像素。为了解决这个问题，我们采用填充(padding)的方法：在图像的边缘填充元素（通常为0）

如果添加p_h行填充和p_w列填充，输出形状变为：

(n_h-k_h+p_h+1)*(n_w-k_w+p_w+1)

很多情况下，会设置p_h=k_h-1，p_w=k_w-1来使输入和输出具有相同的高度和宽度


In [13]:
import torch
from torch import nn

# 为方便起见，我们定义一个计算卷积的函数
# 此函数初始化卷积层权重，并对输入和输出扩大和缩减相应倍数
def comp_conv2d(conv2d,X):
    # 这里的(1,1)表示批量大小和通道数都是1
    X = X.reshape((1,1)+X.shape) # 这里的加号实现的拼接功能
    Y = conv2d(X)
    # 省略前两个维度：批量大小和通道
    return Y.reshape(Y.shape[2:])

# 注意，这里每侧边都填充了1行或1列，因此总共添加了2行或2列
conv2d = nn.Conv2d(1,1,kernel_size=3,padding=1) # 这里kernel_size一般设置为元组，如果是标量的话说明是正方形
X = torch.rand(size=(8,8))
comp_conv2d(conv2d,X).shape

torch.Size([8, 8])

In [14]:
conv2d = nn.Conv2d(1,1,kernel_size=(5,3),padding=(2,1)) # 高度和宽度两侧边的填充分别为2,1
comp_conv2d(conv2d,X).shape

torch.Size([8, 8])

## 6.3.2步幅
有时候为了高效计算或缩减采样次数，卷积窗口可以跳过中间位置，每次滑动多个元素

每次滑动元素的数量称为步幅

当垂直步幅为s_h，水平步幅为s_w时，输出形状为：

[(n_h-k_h+p_h+s_h)/s_h]x[(n_w-k_w+p_w+s_w)/s_w]

可以设置p_h=k_h-1，p_w=k_w-1，输出形状简化为：[(n_h+s_h-1)/s_h]x[(n_w+s_w-1)/s_w]

In [15]:
conv2d = nn.Conv2d(1,1,kernel_size=3,padding=1,stride=2)  # 构建二维卷积层
comp_conv2d(conv2d,X).shape

torch.Size([4, 4])

In [16]:
conv2d = nn.Conv2d(1,1,kernel_size=(3,5),padding=(0,1),stride=(3,4))
comp_conv2d(conv2d,X).shape

torch.Size([2, 2])

实际使用中很少使用不一致的步幅或填充

# 6.4多输入多输出通道
对于RGB输入图像而言，具有3xhxw的形状，这个大小为3的轴称为通道(channel)维度
## 6.4.1多输入通道
当输入包含多个通道时，需要构造一个具有与输入数据相同输入通道数的卷积核，以便于输入数据进行互相关运算。

所谓多输入通道互相关运算，就是对每个通道执行互相关操作

In [20]:
import torch
from d2l import torch as d2l
def corr2d_multi_in(X,K):
    # 先遍历X和K的第0个维度（通道维度），再把它们加到一起
    return sum(d2l.corr2d(x,k) for x,k in zip(X,K))

In [31]:
X = torch.tensor([[[0.0,1.0,2.0],[3.0,4.0,5.0],[6.0,7.0,8.0]],
                  [[1.0,2.0,3.0],[4.0,5.0,6.0],[7.0,8.0,9.0]]])
K = torch.tensor([[[0.0,1.0],[2.0,3.0]],[[1.0,2.0],[3.0,4.0]]])
corr2d_multi_in(X,K)

tensor([[ 56.,  72.],
        [104., 120.]])

## 6.4.2多输出通道
我们可以将每个通道看作对不同特征的响应，每个通道不是独立学习的，而是为了共同使用而优化的。因此，多输出通道并不仅是学习多个单通道的检测器

输入通道数：c_i

输出通道数：c_0

卷积核形状：c_0xc_ixk_hxk_w

In [27]:
def corr2d_multi_in_out(X,K):
    # 迭代K的第0个维度，每次都对输入X执行互相关运算
    # 最后将所有结果都叠加在一起
    return torch.stack([corr2d_multi_in(X,k) for k in K],0)

[corr2d_multi_in(X, k) for k in K]是一个列表推导式，对K中的每个元素执行corr2d_multi_in函数，最终得到一个包含多个卷积结果的列表

torch.stack是PyTorch中的一个函数，用于沿着指定的维度将多个张量堆叠在一起，0表示沿着第0个维度进行堆叠

In [35]:
K = torch.stack((K,K+1,K+2),0)
K.shape

torch.Size([3, 2, 2, 2])

In [36]:
corr2d_multi_in_out(X,K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

## 6.4.31x1卷积层
卷积的本质是有效提取相邻像素间的相关特征，但是1x1卷积没有这个作用

1x1卷积的唯一计算发生在通道上

In [37]:
def corr2d_multi_in_out_1x1(X,K):
    c_i,h,w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i,h*w))
    K = K.reshape((c_o,c_i))
    # 全连接层中的矩阵乘法
    Y = torch.matmul(K,X)
    return Y.reshape((c_o,h,w))

In [38]:
X = torch.normal(0,1,(3,3,3))
K = torch.normal(0,1,(2,3,1,1))
Y1 = corr2d_multi_in_out_1x1(X,K)
Y2 = corr2d_multi_in_out(X,K)
assert float(torch.abs(Y1-Y2).sum())<1e-6

# 6.5汇聚层
通常处理图像时，我们希望逐渐降低隐藏层表示的空间分辨率、聚合信息，这样随着神经网络中层数的增加，每个神经元对其敏感的感受野就越大

汇聚层(pooling layer)具有双重目的：1.降低卷积层对位置的敏感性；2.降低对空间降采样表示的敏感性

## 6.5.1最大汇聚和平均汇聚
最大汇聚(maximum pooling)：计算汇聚窗口中所有元素的最大值；
平均汇聚(average pooling)：计算汇聚窗口中所有元素的平均值

In [41]:
import torch
from torch import nn
from d2l import torch as d2l

def pool2d(X,pool_size,mode='max'):
    p_h,p_w =pool_size
    Y = torch.zeros((X.shape[0] - p_h + 1,X.shape[1] - p_w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i,j] = X[i:i+p_h,j:j+p_w].max()
            elif mode == 'avg':
                Y[i,j] = X[i:i+p_h,j:j+j+p_w].mean()
    return Y

X = torch.tensor([[0.0,1.0,2.0],[3.0,4.0,5.0],[6.0,7.0,8.0]])
pool2d(X,(2,2),'avg')

tensor([[2., 3.],
        [5., 6.]])

## 6.5.2填充和步幅
我们可以通过填充和步幅获得所需要的输出形状

In [42]:
X = torch.arange(16,dtype=torch.float32).reshape((1,1,4,4))
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

In [43]:
pool2d = nn.MaxPool2d(3)
pool2d(X)

tensor([[[[10.]]]])

In [44]:
pool2d = nn.MaxPool2d(3,padding=1,stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

默认情况下，深度学习框架中的步幅与汇聚窗口的大小相同。因此，如果使用形状为(3,3)的汇聚窗口，那么默认情况下，步幅形状为(3,3)

In [45]:
pool2d = nn.MaxPool2d((2,3),stride=(2,3),padding=(0,1))
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

## 6.5.3多个通道
在处理多通道输入数据时，汇聚层在每个输入通道上单独运算，而不是像卷积层那样在通道上对输入进行汇总（结合PPT理解一下吧）

这意味着汇聚层的输出通道数与输入通道数相同

以下代码在通道维度（第一维）连接张量X和X+1，以构建具有2个通道的输入

In [46]:
X = torch.cat((X,X+1),1)
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])

In [47]:
pool2d = nn.MaxPool2d(3,padding=1,stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])

training on cuda:0


In [1]:
print("hello")

hello
